In [ ]:
# Medical TokAlign on RunPod (H100)
End-to-end: setup -> data prep -> tokenizer adaptation -> evaluation.


In [ ]:
import torch, os
print('CUDA available:', torch.cuda.is_available())
print('CUDA device:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU')
os.environ.setdefault('CUDA_VISIBLE_DEVICES','0')
os.environ.setdefault('PYTORCH_CUDA_ALLOC_CONF','expandable_segments:True')
os.environ.setdefault('HF_HOME','/workspace/.cache/huggingface')
# RunPod often sets HUGGINGFACE_TOKEN; alias to HF_TOKEN for libraries
if 'HUGGINGFACE_TOKEN' in os.environ and 'HF_TOKEN' not in os.environ:
    os.environ['HF_TOKEN'] = os.environ['HUGGINGFACE_TOKEN']
    os.environ['HUGGINGFACE_HUB_TOKEN'] = os.environ['HUGGINGFACE_TOKEN']
print('HF_HOME:', os.environ['HF_HOME'])
print('HF_TOKEN set:', 'HF_TOKEN' in os.environ)


In [ ]:
# Install core requirements
!pip install -r medical_tokalign/requirements.txt


In [ ]:
# Prepare datasets (standard splits)
!bash medical_tokalign/scripts/prepare_medical_data.sh


In [ ]:
# Run tokenizer adaptation (TokAlign-style full adaptation)
!bash medical_tokalign/scripts/run_vocab_adaptation.sh --model_id meta-llama/Meta-Llama-3.1-8B --top_k 8192


In [ ]:
# Evaluate (vLLM backend by default)
!bash medical_tokalign/scripts/eval_medical.sh --config medical_tokalign/configs/eval_medical.yaml


In [ ]:
# Display latest metrics
import os, glob, json
runs = sorted(glob.glob('medical_tokalign/runs/medical_eval/*'))
print('Runs:', runs[-3:])
if runs:
    path = os.path.join(runs[-1], 'metrics.json')
    print('Latest metrics:', path)
    print(json.dumps(json.load(open(path)), indent=2))
